In [ ]:
class StopExecution(Exception):
    def _render_traceback_(self):
        return []

In [ ]:
# Starting from Python 3.8 DLL search policy has changed.
# We need to add path to CUDA DLLs explicitly.
import os

if os.name == "nt":
    # Add CUDA_PATH env variable
    cuda_path = os.environ["CUDA_PATH"]
    if cuda_path:
        os.add_dll_directory(os.path.join(cuda_path, "bin"))
    else:
        raise StopExecution

In [ ]:
from PIL import Image
from IPython.display import display

import python_vali as vali
import numpy as np

In [ ]:
url = "../tests/data/test.mp4"

In [ ]:
# GPU-accelerated decoder
pyDec = vali.PyDecoder(
    url,
    {},
    gpu_id=0)

# GPU-accelerated converter
pyCvt = vali.PySurfaceConverter(
    pyDec.Format,
    vali.PixelFormat.RGB,
    gpu_id=0)

# GPU-accelerated Surface downloader
pyDwn = vali.PySurfaceDownloader(gpu_id=0)

# GPU-accelerated Surface rotator
pyRot = vali.PySurfaceRotator(gpu_id=0)

In [ ]:
def calc_rotation(
        angle: int,
        surf_src: vali.Surface):

    shift_x = shift_y = 0
    angle = (angle + 360) % 360

    if angle == 0:
        pass
    elif angle == 90:
        shift_y = surf_src.Width
    elif angle == 180:
        shift_x = surf_src.Width
        shift_y = surf_src.Height
    elif angle == 270:
        shift_x = surf_src.Height

    return (angle * 1.0, shift_x * 1.0, shift_y * 1.0)

In [ ]:
# Raw decoded Surface
surf_src = vali.Surface.Make(
    format=pyDec.Format,
    width=pyDec.Width,
    height=pyDec.Height,
    gpu_id=0)

# Raw Surface, converted to RGB
surf_dst = vali.Surface.Make(
    format=vali.PixelFormat.RGB,
    width=pyDec.Width,
    height=pyDec.Height,
    gpu_id=0)

# Rotated Surface
surf_rot = vali.Surface.Make(
    format=vali.PixelFormat.RGB,
    width=surf_dst.Height,
    height=surf_dst.Width,
    gpu_id=0)

# Numpy array which contains decoded RGB Surface
frame = np.ndarray(
    dtype=np.uint8,
    shape=(surf_rot.Height, surf_rot.Width, 3))

for _ in range(0, 2):
    # Decode single Surface
    success, details = pyDec.DecodeSingleSurface(surf_src)
    if not success:
        print(details)
        raise StopExecution

    # Convert to RGB
    success, details = pyCvt.Run(surf_src, surf_dst)
    if not success:
        print(details)
        raise StopExecution
      
    # Rotate
    success, details = pyRot.Run(
        angle=270.0, 
        shift_x=0.0,
        shift_y=0.0,
        src=surf_dst, 
        dst=surf_rot)
    if not success:
        print(details)
        raise StopExecution

    # Copy pixels to numpy ndarray
    success, details = pyDwn.Run(surf_rot, frame)
    if not success:
        print(details)
        raise StopExecution

    # Display in notebook
    display(Image.fromarray(frame), display_id="decoded_frame")